In [ ]:
#| output: false
folder = "26z-thin-debug-highCCScost"
import os
from typing import Dict, List

import pandas as pd
import altair as alt
from pathlib import Path
from fig_functions import (
    fix_tx_line_names,
    load_genx_operations_data,
    load_data,
    chart_total_cap,
    chart_regional_cap,
    chart_total_gen,
    chart_regional_gen,
    chart_tx_expansion,
    chart_emissions,
    chart_dispatch,
    chart_wind_dispatch,
    chart_op_cost,
    chart_op_nse
)

alt.renderers.enable("html")
alt.data_transformers.disable_max_rows()

cwd = Path.cwd()
data_path = cwd.parent / folder



fig_num = 0

cap = load_data(data_path, "resource_capacity.csv")
cap = cap.query("unit=='MW' and not tech_type.isna()")
cap["additions"] = cap["end_value"] - cap["start_value"]

gen = load_data(data_path, "generation.csv")

tx = load_data(data_path, "transmission.csv")


tx["start_region"] = tx["line_name"].str.split("_to_").str[0]
tx["dest_region"] = tx["line_name"].str.split("_to_").str[1]

first_year = tx["planning_year"].min()
starting_tx = tx.loc[tx["planning_year"] == first_year, :]
starting_tx = starting_tx.rename(columns={"start_value": "value"})
starting_tx["planning_year"] = 2023

tx_exp = load_data(data_path, "transmission_expansion.csv")
tx_exp["start_region"] = tx_exp["line_name"].str.split("_to_").str[0]
tx_exp["dest_region"] = tx_exp["line_name"].str.split("_to_").str[1]

emiss = load_data(data_path, "emissions.csv")

emiss.loc[emiss["unit"] == "kg", "value"] /= 1000


dispatch = load_data(data_path, "dispatch.csv")

dispatch = dispatch.groupby(
    [
        "planning_year",
        "model",
        "agg_zone",
        "zone",
        "tech_type",
        "resource_name",
        "hour",
    ],
    as_index=False,
)["value"].sum()
group_cols = ["planning_year", "model", "agg_zone", "zone", "tech_type"]
hours = dispatch["hour"].unique()
index_cols = ["resource_name"]
df_list = []
for _, _df in dispatch.groupby(group_cols):
    multi_index = pd.MultiIndex.from_product(
        [_df[col].unique() for col in index_cols] + [hours],
        names=index_cols + ["hour"],
    )
    _df = _df.set_index(index_cols + ["hour"])
    _df = _df.reindex(index=multi_index, fill_value=0)
    _df = _df.reset_index()
    for val, col in zip(_, group_cols):
        _df[col] = val
    df_list.append(_df)

dispatch = pd.concat(df_list, ignore_index=True)

# Capacity and Generation

## Row

#### Column {.tabset}

In [ ]:
chart = chart_total_cap(cap)
chart

In [ ]:
chart = chart_regional_cap(cap)
chart

#### Column {.tabset}

In [ ]:
chart = chart_total_gen(gen, cap)
chart

In [ ]:
chart = chart_regional_gen(gen, cap)
chart

# Transmission Expansion
## Row

In [ ]:
data = pd.concat([starting_tx, tx_exp])
chart = chart_tx_expansion(data)
chart

# Emissions

In [ ]:
chart = chart_emissions(emiss)
chart

# Dispatch

## Row {.tabset}

In [ ]:
data = dispatch.query("planning_year==2030").groupby(["model", "tech_type", "agg_zone", "hour"], as_index=False)[
    "value"
].sum()
chart = chart_dispatch(data)
chart

In [ ]:
data = dispatch.query("planning_year==2040").groupby(["model", "tech_type", "agg_zone", "hour"], as_index=False)[
    "value"
].sum()
chart = chart_dispatch(data)
chart

In [ ]:
data = dispatch.query("planning_year==2050").groupby(["model", "tech_type", "agg_zone", "hour"], as_index=False)[
    "value"
].sum()
chart = chart_dispatch(data)
chart

In [ ]:
wind_dispatch = dispatch.query("resource_name.str.contains('landbasedwind')")
wind_dispatch["cluster"] = wind_dispatch["resource_name"].str.split("_").str[-1]
data = wind_dispatch.query("planning_year==2030").groupby(["model", "tech_type", "cluster", "zone", "hour"], as_index=False)[
    "value"
].sum()
chart = chart_wind_dispatch(data)
chart

In [ ]:
data = wind_dispatch.query("planning_year==2040").groupby(["model", "tech_type", "cluster", "zone", "hour"], as_index=False)[
    "value"
].sum()
chart = chart_wind_dispatch(data)
chart

In [ ]:
data = wind_dispatch.query("planning_year==2050").groupby(["model", "tech_type", "cluster", "zone", "hour"], as_index=False)[
    "value"
].sum()
chart = chart_wind_dispatch(data)
chart

# Operational Results
## Row {.tabset}

In [ ]:
op_costs = load_genx_operations_data(data_path, "costs.csv")
chart = chart_op_cost(op_costs)
chart

In [ ]:
op_nse = load_genx_operations_data(data_path, "nse.csv")
chart = chart_op_nse(op_nse)
chart